In [1]:
import torch
import pkg_resources
pkg_resources.require("pandas==1.5.3")
import pandas as pd

BASE_PATH = '/content/drive/MyDrive/Generative_ML/current_data/' #@param {type:"string"}
BASE_PATH = '/Users/morgunov/batista/Summer/pipeline/' #@param {type:"string"}
TRAINING_PATH = BASE_PATH + 'raw_data/combined_train.csv' #@param {type:"string"}
VALIDATION_PATH = BASE_PATH + 'raw_data/combined_val.csv' #@param {type:"string"}
CHECKPOINTS_PATH = BASE_PATH + 'checkpoints/' #@param {type:"string"}
PICKLES_PATH = BASE_PATH + 'pickles/' #@param {type:"string"}
INFERENCE_PATH = BASE_PATH + 'inferences/' #@param {type:"string"}
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
REGEX_PATTERN = "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|!|\*|\$|\%[0-9]{2}|[0-9])" #@param {type:"string"}

date, time = str(pd.to_datetime('today', utc=True)).split()
FILE_SUFFIX = '_'+'_'.join(date.split('-')[1:]) + '_' + ':'.join(time.split(':')[:2]) # @param {type:"string"}

'mps'

In [2]:
CONFIG_DICT = {
    "device": DEVICE,
    "att_bias": False,
    "gpt_bias": True,
    "att_drop_rate": 0.1,
    "gpt_drop_rate": 0.1,
    "n_layer": 8,
    "n_head": 8,
    "n_embed": 256,
    "ff_mult": 4, # multiplier for Feed Forward number of hidden units inside multihead,
    "doGELU": True, # else ReLU
    "attention_times": [],
    "do_flash": True,
    "desc_path": CHECKPOINTS_PATH + f'smiles_dataset_attributes{FILE_SUFFIX}.yaml',
    "ckpt_path": CHECKPOINTS_PATH + f'GPT_pretrain{FILE_SUFFIX}.pt',
    "train_path": TRAINING_PATH,
    "val_path": VALIDATION_PATH,
    "wandb_project": f'GPT_pretrain_pipeline',
    "wandb_runname": f'GPT_pretrain{FILE_SUFFIX}',
    "slice_data": 10_000, #False for all data
    "epochs": 2,
    "batch_size": 16, #512,
    "learning_rate": 3e-4,
    "betas": (0.965, 0.99), #(0.9, 0.95)
    "rho": 0.04, # For SophiaG
    "weight_decay": 0.1,
    "lr_decay": False,
    "num_workers": 0 # number of worker processes to use for loading data
}

# GPT

## Imports & Installations

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# install necessary packages
!pip install rdkit
!pip install pandas==1.5.3
!pip install molsets
!pip install wandb

# clone Sophia optimizer GitHub repository
!git clone https://github.com/Liuhong99/Sophia.git

In [3]:
# import necessary packages
import numpy as np
import os
import re
import logging
import wandb
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import torch.optim as optim
from torch.cuda.amp import GradScaler
from tqdm import tqdm
from rdkit import Chem
import moses
from moses.utils import get_mol
from Sophia.sophia import SophiaG
import yaml

# set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

/Users/morgunov/.pyenv/versions/3.11.4/lib/python3.11/site-packages/molsets-1.0-py3.11.egg/moses/metrics/utils.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  _mcf.append(_pains, sort=True)['smarts'].values]


## Definitions

In [4]:
class GPTConfig:
    def __init__(self, vocab_size=None, block_size=None, **kwargs):
        self.vocab_size = vocab_size
        self.block_size = block_size
        for k,v in kwargs.items():
            setattr(self, k, v)

    def export_attributes(self, export_path):
        with open(export_path, 'w') as f:
            yaml.dump(vars(self), f)

    def load_attributes(self, load_path):
        with open(load_path, 'r') as f:
            config_dict = yaml.load(f, Loader=yaml.SafeLoader)
        self.__dict__.update(config_dict)

### Dataset loading & sampling

In [5]:
@torch.no_grad()
def sample(model, x, steps, temperature=1.0):
    block_size = model.get_block_size() # define size of context window used for input conditioning
    model.eval()
    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # limit conditioning input to the most recent block_size elements
        logits, _= model(x_cond) # give input to model and get logits (unnormalized scores or probabilities)
        logits = logits[:, -1, :] / temperature # extract the logits for the next token in the sequence
        probs = F.softmax(logits, dim=-1)
        ix = torch.multinomial(probs, 1)
        x = torch.cat((x, ix), dim=1) # concatenate the chosen token index with the existing sequence
    return x

def check_novelty(gen_smiles, train_smiles):
    if len(gen_smiles) == 0:
        novel_ratio = 0
    else:
        duplicates = [1 for mol in gen_smiles if mol in train_smiles]
        novel = len(gen_smiles) - sum(duplicates)
        novel_ratio = novel*100/len(gen_smiles)
    return novel_ratio

def canonic_smiles(smiles_or_mol):
    mol = get_mol(smiles_or_mol)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol)


class SMILESDataset(Dataset):
    
    def __init__(self, data=None, chars=None, block_size=None, len_data = None):
        if chars is None:
            self.desc_only = True
            return
        self.desc_only = False

        self.vocab_size = len(chars)
        self.stoi = {ch:i for i,ch in enumerate(chars)}
        self.itos = {i:ch for i,ch in enumerate(chars)}

        self.block_size = block_size
        self.data = data
        self.len_data = len_data

    def export_desc_attributes(self, export_path):
        attr_dict = {
            "desc_only": self.desc_only,
            "vocab_size": self.vocab_size,
            "block_size": self.block_size,
            "stoi": self.stoi,
            "itos": self.itos,
            "len_data": self.len_data
        }
        with open(export_path, 'w') as f:
            yaml.dump(attr_dict, f)

    def load_desc_attributes(self, load_path):
        with open(load_path, 'r') as f:
            attr_dict = yaml.load(f, Loader=yaml.SafeLoader)
        self.__dict__.update(attr_dict)

    def __len__(self):
        assert not self.desc_only, "Dataset is not initialized"
        return len(self.data)

    def __getitem__(self, idx):
        assert not self.desc_only, "Dataset is not initialized"
        smiles = self.data[idx].strip()
        # define regular expressin pattern used to identify characters in the SMILES strings
        regex = re.compile(REGEX_PATTERN)
        smiles_matches = regex.findall(smiles)

        # smiles = str('!') + smiles
        # smiles += str('<')*(self.block_size - len(smiles_matches)) # pad SMILES string by appending '<' to the end until self.block_size is achieved
        
        if len(smiles_matches) > self.block_size+1: # if the number of matches found by the regular expression pattern applied to the SMILES string exceeds self.block_size:
            smiles = smiles[:self.block_size+1]
        
        embedded_smile = [self.stoi[s] for s in smiles_matches]
        x = torch.tensor(embedded_smile[:-1], dtype=torch.long)
        y = torch.tensor(embedded_smile[1:], dtype=torch.long)
        return x, y


### Model

In [6]:
class SelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embed % config.n_head == 0
        self.config = config

        self.query = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)
        self.key = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)
        self.value = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)

        self.attn_drop = nn.Dropout(config.att_drop_rate)
        self.resid_drop = nn.Dropout(config.att_drop_rate)

        self.proj = nn.Linear(config.n_embed, config.n_embed)
        self.n_head = config.n_head

        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size))
                                .view(1, 1, config.block_size, config.block_size))

    def forward(self, x, layer_past=None):
        B, T, C = x.size()
        # apply attention functions to get tensors with dimensions (B, n_head, T, head_size)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head)
        k = self.key(x).view(B, T, self.n_head, C // self.n_head)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head)
        if self.config.do_flash:
            q = q.transpose(1, 2)
            k = k.transpose(1, 2)
            v = v.transpose(1, 2)
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, dropout_p=self.config.att_drop_rate if self.training else 0, is_causal=True)
            y = y.transpose(1, 2)
        else:
            # (B h T s) @ (B h s T) -> (B h T T)
            att = torch.einsum('bths,bihs->bhti', q, k) / np.sqrt(k.size(-1))
            att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            # (B h T T) @ (B h T s) -> (B h T s)
            y = torch.einsum('bhtq,bqhs->bths', att, v)
            self.att_weights = att
        self.attended = y
        y = y.contiguous().view(B, T, C)
        y = self.resid_drop(self.proj(y))
        self.out = y
        return y

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embed)
        self.ln2 = nn.LayerNorm(config.n_embed)
        self.attn = SelfAttention(config)
        self.mlp = nn.Sequential(nn.Linear(config.n_embed, config.ff_mult*config.n_embed), nn.GELU() if config.doGELU else nn.ReLU(),
            nn.Linear(config.ff_mult*config.n_embed, config.n_embed), nn.Dropout(config.att_drop_rate))

    def forward(self, x):
        y = self.attn(self.ln1(x))
        x = x + y # perform a residual connection by summing input and attention output
        x = x + self.mlp(self.ln2(x)) # apply layer normalization and then MLP, create a residual connection with input
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embed)
        self.type_emb = nn.Embedding(2, config.n_embed)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embed))

        self.drop = nn.Dropout(config.gpt_drop_rate)
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])

        self.ln_f = nn.LayerNorm(config.n_embed)
        self.head = nn.Linear(config.n_embed, config.vocab_size, bias=config.gpt_bias)
        self.block_size = config.block_size # define the context size
        self.apply(self._init_weights) # initialize weights and apply to all relevant modules in the model

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        decay, no_decay = set(), set()
        no_decay = set()

        whitelist_weight_modules = (torch.nn.Linear)
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        # for named module of the model:
        for mn, m in self.named_modules():
            # for named parameter of each module:
            for pn, p in m.named_parameters():
                # construct full parameter name by concatenating module name and parameter name, separated by a dot
                fpn = '%s.%s' % (mn, pn) if mn else pn
                if pn.endswith('bias') or ('bias' in pn):
                    no_decay.add(fpn)
                elif (pn.endswith('weight') or ('weight' in pn)) and isinstance(m, whitelist_weight_modules):
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    no_decay.add(fpn)
        no_decay.add('pos_emb')
        param_dict = {pn:p for pn, p in self.named_parameters()}
        assert len(decay & no_decay) == 0
        # assert that all parameters from both sets have been correctly separated
        assert len(param_dict.keys() - (decay | no_decay)) == 0
        optim_groups = [{"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
                        {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0}]
        optimizer = SophiaG(optim_groups, lr=train_config.learning_rate, betas=train_config.betas, rho=train_config.rho, weight_decay=train_config.weight_decay)
        return optimizer

    def forward(self, idx, targets=None, prop = None, scaffold = None):
        b, t = idx.size()

        assert t <= self.block_size

        token_embeddings = self.tok_emb(idx) # pass input tensor through token embedding layer
        # select a subset of the position embedding matrix based on the length of the input sequence
        position_embeddings = self.pos_emb[:, :t, :]
        # pass a tensor of ones of shape (b, t) through the type embedding layer,
        # maps a binary type indicator to a learnable embedding vector, all type indicators
        # are set to 1, indicating same type for all tokens in input sequence
        type_embeddings = self.type_emb(torch.ones((b,t), dtype=torch.long, device=idx.device))
        x = self.drop(token_embeddings + position_embeddings + type_embeddings)
        
        for layer in self.blocks:
            x = layer(x)
        x = self.ln_f(x)
        logits = self.head(x)
        loss = F.cross_entropy(logits.reshape(-1, logits.size(-1)), targets.view(-1)) if targets is not None else None
        return logits, loss

### Training

In [12]:
class Trainer:

    def __init__(self, model, train_dataset, test_dataset):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = model.config
        self.stoi = train_dataset.stoi
        self.itos = train_dataset.itos

    def train(self, wandb):
        model, config = self.model, self.config
        optimizer = model.configure_optimizers(config)
        scaler = GradScaler() # define variable used for gradient scaling in mixed-precision training
        self.tokens = 0 # initialize a counter used for learning rate decay

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True, batch_size=config.batch_size, num_workers=config.num_workers)
            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            # for batch index, batch in progress bar:
            for it, (x, y) in pbar:
                # move the input data tensor, target data tensor, property tensor, and scaffold tensor to GPU
                x, y = x.to(config.device), y.to(config.device)
                # allow model to use lower-precision computations for improved memory usage
                if config.device == 'cuda':
                    with torch.cuda.amp.autocast():
                        with torch.set_grad_enabled(is_train):
                            logits, loss = model(x, y)
                            loss = loss.mean()
                            losses.append(loss.item())
                else:
                    with torch.cpu.amp.autocast():
                        with torch.set_grad_enabled(is_train):
                            logits, loss = model(x, y)
                            loss = loss.mean()
                            losses.append(loss.item())

                if is_train:
                    model.zero_grad()
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer) # unscale the gradients of the optimizer's parameters to their original values
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients of model parameters to prevent them from exploding, setting maximum gradient norm to be 1.0
                    scaler.step(optimizer) # update the optimizer's parameters based on calculated gradients
                    scaler.update() # update the scale factor of the gradient scaler
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # increment the number of processed tokens by the count of valid tokens (not padding or special tokens)
                        if self.tokens < config.warmup_tokens:
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens)) # perform a linear warm-up
                        else:
                            # calculate the progress of training in terms of the number of tokens processed
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            # calculate the scaling factor for the learning rate (between 0.1 and 1.0)
                            # to gradually reduce learning rate as training progresses
                            lr_mult = max(0.1, 0.5 * (1.0 + np.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult # multiply the base learning rate by the scaling factor to obtain the updated learning rate
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate
                    if wandb is not None: # log training progress using Weights & Biases
                        wandb.log({'step_train_loss': loss, 'train_step': it + epoch*len(loader), 'learning_rate': lr})
                    # update the description of the progress bar with epoch, iteration, and training loss
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
            return float(np.mean(losses))

        # initialize best loss as infinity
        best_loss = float('inf')
        for epoch in range(config.epochs):
            print(f'{epoch=}')
            train_loss = run_epoch('train')
            if self.test_dataset is not None:
                test_loss = run_epoch('test')
            if wandb is not None:
                wandb.log({'epoch_valid_loss': test_loss, 'epoch_train_loss': train_loss, 'epoch': epoch + 1})
            good_model = self.test_dataset is None or test_loss < best_loss
            if good_model:
                best_loss = test_loss
                torch.save(self.model.state_dict(), self.config.ckpt_path)

In [8]:
def load_data(config_dict):
    if (cut:=config_dict["slice_data"]):
        train_data = pd.read_csv(config_dict["train_path"])[:cut]
        val_data = pd.read_csv(config_dict["val_path"])[:cut]
    else:
        train_data = pd.read_csv(config_dict["train_path"])
        val_data = pd.read_csv(config_dict["val_path"])

    smiles = train_data['smiles']
    vsmiles = val_data['smiles']

    # compile pattern into a regular expression object that can be used for matching operations
    regex = re.compile(REGEX_PATTERN)
    char_set = {'<','!'} # context={'<'}

    max_len = 0
    for iterator in (smiles.values, vsmiles.values):
        for i in iterator:
            chars = regex.findall(i.strip())
            max_len = max(max_len, len(chars))
            for char in chars:
                char_set.add(char)

    chars = sorted(list(char_set))
    max_len += 1    #accounting for the start token, which hasn't been added yet

    smiles = ['!' + i + str('<')*(max_len - len(regex.findall(i.strip()))) for i in smiles]
    vsmiles = ['!' + i + str('<')*(max_len - len(regex.findall(i.strip()))) for i in vsmiles]

    train_dataset = SMILESDataset(data=smiles, chars=chars, block_size=max_len, len_data=len(train_data))
    valid_dataset = SMILESDataset(data=vsmiles, chars=chars, block_size=max_len, len_data=len(val_data))
    train_dataset.export_desc_attributes(config_dict["desc_path"])
    return train_dataset, valid_dataset

In [13]:
def pretrain_GPT(train_dataset, valid_dataset, config_dict):
  """
  OUTPUTS:
  1) checkpoint of trained model parameters
  2) Weights & Biases logged run
  """

  mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, warmup_tokens=0.1*train_dataset.len_data*train_dataset.block_size, final_tokens=config_dict["epochs"]*train_dataset.len_data*train_dataset.block_size, **config_dict)
  model = GPT(mconf)
  model.to(config_dict["device"])
  torch.compile(model)
  trainer = Trainer(model, train_dataset, valid_dataset)

  %env WANDB_EXECUTABLE=python3
  wandb.init(project=config_dict["wandb_project"], name=config_dict["wandb_runname"])
  trainer.train(wandb=wandb)
  return model, tconf

In [10]:
train_dataset, val_dataset = load_data(CONFIG_DICT) # takes ~1 min

In [ ]:
# define function to train GPT model
model, tconf = pretrain_GPT(
                train_dataset = train_dataset,
                valid_dataset = val_dataset,
                config_dict = CONFIG_DICT
        )

#GK wandb API Key: c99c9a01523f93287716691fa3360b1f4566e115
#RB wandb API Key: 4d3d628c6b5a4b3554c7a89ea50df8a4a6be0f85
#AM wandb API key: 5be14d5930441de4707f6a58e4f7c2e229dab1d1

##Generation & Evaluation

In [ ]:
def generate_SMILES(model_config_dict, inference_config_dict):
  props = inference_config_dict["props"]
  scaffold = inference_config_dict["scaffold"]

  # define a regular expression that matches molecular tokens in SMILES strings
  pattern = "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|!|\*|\$|\%[0-9]{2}|[0-9])"
  regex = re.compile(pattern)

  dataset = SMILESDataset()
  dataset.load_desc_attributes(inference_config_dict['desc_path'])
  use_scaf = False if scaffold is None else True

  mconf = GPTConfig(dataset.vocab_size, dataset.max_len, num_props=len(props), scaffold=use_scaf, scaffold_maxlen=dataset.scaf_max_len, **model_config_dict)
  model = GPT(mconf).to(model_config_dict['device'])
  torch.compile(model)

  # load parameters into the model
  model.load_state_dict(torch.load(inference_config_dict["model_params"], map_location=torch.device(model_config_dict['device'])))
  block_size = model.get_block_size() #inference_config_dict["block_size"]
  assert block_size == dataset.max_len, "Warning: model block size and dataset block size are different"
  # calculate number of generation iterations from total number of SMILES to generate and batch size
  gen_iter = math.ceil(inference_config_dict["gen_size"] / inference_config_dict["batch_size"])
  stoi = dataset.stoi # define dictionary to map strings to integers
  itos = dataset.itos # define dictionary to map integers to strings
  # is a scaffold is defined for conditioning:
  if scaffold is not None:
      # pad '<' to end of scaffold string to achieve maximum scaffold length
      scaffold += str('<')*(dataset.scaf_max_len - len(regex.findall(scaffold)))
      # convert the scaffold SMILES to a tensor of integers and repeat along the batch dimension, move to GPU
      scaffold=torch.tensor([stoi[s] for s in regex.findall(scaffold)])[None,...].repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])

  if props is None:
    p = None
  elif len(props) == 1:
    # create a tensor for conditioning with a single property value
    p = torch.tensor([[props[0]]]).repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])
  else:
    # create a tensor for conditioning with multiple property values
    p = torch.tensor([props]).repeat(inference_config_dict["batch_size"], 1).unsqueeze(1).to(model_config_dict['device'])

  molecules = []
  for i in tqdm(range(gen_iter)):
          # create an input tensor by converting 'context' to a tensor of token indices,
          # repeat this batch times along the batch dimension
          x = torch.tensor([stoi[s] for s in regex.findall(inference_config_dict["context"])], dtype=torch.long)[None,...].repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])
          # call sample function to generate molecules conditioned on the input
          y = sample(model, x, block_size, temperature=inference_config_dict["temp"], prop=p, scaffold=scaffold)
          # for each generated molecule:
          for gen_mol in y:
                  # convert generated molecule from list of integers to list of strings and concatenate to one string
                  completion = ''.join([itos[int(i)] for i in gen_mol])
                  # remove padding tokens
                  completion = completion.replace('<', '')
                  completion = completion.replace('!', '')
                  # convert the string representation of the molecule to an rdkit Mol object
                  mol = get_mol(completion)
                  # if an rdkit Mol object was created:
                  if mol:
                          # append the Mol object to the list
                          molecules.append(mol)
  # create dataframe where first column contains rdkit Mols and second column contins SMILES
  results = pd.DataFrame([{'molecule' : i, 'smiles': Chem.MolToSmiles(i)} for i in molecules])
  # iterate over each SMILES and ensure that equivalent molecules have same SMILES
  canon_smiles = [canonic_smiles(s) for s in results['smiles']]
  # create set of unique SMILES strings
  unique_smiles = list(set(canon_smiles))
  data = pd.read_csv(inference_config_dict["train_data"]) # load training data
  novel_ratio = check_novelty(unique_smiles, set(data['smiles'])) # calculate novelty ratio from generated SMILES and training SMILES
  results['qed'] = results['molecule'].apply(lambda x: QED.qed(x)) # quantitative estimate of drug-likeliness (QED)
  results['sas'] = results['molecule'].apply(lambda x: sascorer.calculateScore(x)) #synthetic accessibility score (SAS)
  results['logp'] = results['molecule'].apply(lambda x: Crippen.MolLogP(x)) #(measure of hydrophobicity)
  results['tpsa'] = results['molecule'].apply(lambda x: CalcTPSA(x)) #topological polar surface area (TPSA)
  results['validity'] = np.round(len(results)/(inference_config_dict["batch_size"]*gen_iter), 3)
  results['unique'] = np.round(len(unique_smiles)/len(results), 3)
  results['novelty'] = np.round(novel_ratio/100, 3)
  # save the dataframe as a csv file
  results.to_csv(inference_config_dict["save_path"], index = False)
  # print all evaluation metrics using function from moses package
  print(moses.get_all_metrics(list(results['smiles'].values), device=model_config_dict['device']))

In [ ]:
inference_config_dict = {
    # "model_params": train_config_dict["ckpt_path"],
    "model_params": BASE + 'current_data/GPT_pretrain_07-06.pt',
    "train_data": train_config_dict["train_path"],
    # "desc_path": model_config_dict["desc_path"],
    "desc_path": BASE + 'checkpoints/descriptors_10k.yaml',
    "save_path": BASE + 'current_data/GPT_pretrain_inference_07-06.csv',
    "checkpoint_dir": BASE + 'current_data/',
    "batch_size": 1,
    "gen_size": 10000,
    "temp": 1,
    "context": "!",
    "scaffold": None,
    "props": []
}

In [ ]:
# run function to generate SMILES strings
generate_SMILES(
              model_config_dict = model_config_dict,
              inference_config_dict = inference_config_dict
              )

#Sampling for DiffDock

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install rdkit

In [ ]:
import rdkit.Chem
import rdkit.Chem.Descriptors
from tqdm import tqdm
import pandas as pd

def descriptors_for_gpt_predictions(path_to_predicted, path_to_save):
    gpt_mols = pd.read_csv(path_to_predicted)
    keySet = None
    keyToData = {}
    pbar = tqdm(gpt_mols.iterrows(), total=len(gpt_mols))
    for index, row in pbar:
        smile = row['smiles']
        mol = rdkit.Chem.MolFromSmiles(smile)
        if not mol: continue
        mol_data = rdkit.Chem.Descriptors.CalcMolDescriptors(mol)
        if keySet is None:
            keySet = set(mol_data.keys())
        for key in keySet:
            keyToData.setdefault(key, []).append(mol_data[key])
        keyToData.setdefault('smiles', []).append(smile)
    gpt_df = pd.DataFrame(keyToData)
    gpt_df.to_pickle(path_to_save)
    return gpt_df

descriptors_for_gpt_predictions(path_to_predicted=inference_config_dict['save_path'], path_to_save=inference_config_dict['save_path'].split('.')[0]+'_descriptors.pickle')

In [ ]:
import pickle
import pandas as pd

def project_into_pca_space(path_to_pca, path_to_mols):
    scaler, pca = pickle.load(open(path_to_pca, 'rb'))
    gptMols = pd.read_pickle(path_to_mols)
    return gptMols['smiles'], pca.transform(scaler.transform(gptMols[scaler.get_feature_names_out()]))

gpt_smiles, pca_transformed = project_into_pca_space(path_to_pca=inference_config_dict['checkpoint_dir'] + 'scaler_pca_tuple.pickle', path_to_mols=inference_config_dict['save_path'].split('.')[0]+'_descriptors.pickle')

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from tqdm import tqdm

def _cluster_mols_experimental_loss(mols, n_clusters, n_iter):
    min_loss, best_kmeans = float('inf'), None
    for _ in range(n_iter):
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto', init='k-means++').fit(mols)
        if kmeans.inertia_ < min_loss:
            min_loss = kmeans.inertia_
            best_kmeans = kmeans
    return best_kmeans

def _cluster_mols_experimental_variance(mols, n_clusters, n_iter):
    max_variance, best_kmeans = float('-inf'), None
    for _ in range(n_iter):
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto', init='k-means++').fit(mols)
        counts = np.unique(kmeans.labels_, return_counts=True)[1]
        if (variance:=np.var(counts)) > max_variance:
            max_variance = variance
            best_kmeans = kmeans
    return best_kmeans

def _cluster_mols_experimental_mixed(mols, n_clusters, n_iter, mixed_objective_loss_quantile):
    inertias = []
    variances = []
    km_objs = []
    for _ in range(n_iter):
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto', init='k-means++').fit(mols)
        inertias.append(kmeans.inertia_)
        counts = np.unique(kmeans.labels_, return_counts=True)[1]
        variances.append(np.var(counts))
        km_objs.append(kmeans)
    loss_var_kmeans_triples = sorted(zip(inertias, variances, km_objs), key=lambda x: x[0])
    lowest_n = loss_var_kmeans_triples[:int(len(loss_var_kmeans_triples) * mixed_objective_loss_quantile)]
    sorted_by_variance = sorted(lowest_n, key=lambda x: x[1])
    return sorted_by_variance[0][2]

def _cluster_mols_experimental(mols, n_clusters, save_path, n_iter=1, objective='loss', mixed_objective_loss_quantile=0.1):
    if n_iter == 1:
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto', init='k-means++').fit(mols)
    elif objective == 'loss':
        kmeans = _cluster_mols_experimental_loss(mols, n_clusters, n_iter)
    elif objective == 'variance':
        kmeans = _cluster_mols_experimental_variance(mols, n_clusters, n_iter)
    elif objective == 'mixed':
        kmeans = _cluster_mols_experimental_mixed(mols, n_clusters, n_iter, mixed_objective_loss_quantile)
    else:
        raise ValueError(f'Unknown objective {objective}')

    pickle.dump(kmeans, open(save_path, 'wb'))
    return kmeans

# out = _cluster_mols_experimental(mols=pca_transformed, n_clusters=100, n_iter=1_000)

In [ ]:
from sklearn.cluster import KMeans
from pprint import pprint as pp
import numpy as np
import random

def cluster_and_sample(mols, mols_smiles, n_clusters, n_samples, kmeans_save_path, clusters_save_path, ensure_correctness=False, path_to_pca=None):
    """
        Clusters a given list of molecules, samples from each cluster, and saves the resulting data to specified files.

        This function performs K-Means clustering on the input list of molecules and then samples a specified number of molecules
        from each cluster. The function ensures that the number of samples requested from each cluster doesn't exceed the total number
        of available molecules. The clustered data and sampled data are saved to specified file paths using pickle.

        Parameters
        ----------
        mols : array-like or sparse matrix, shape (n_samples, n_features)
            The input samples where n_samples is the number of samples and n_features is the number of features.

        mols_smiles : list of str
            A list of SMILES strings corresponding to the input molecules.

        n_clusters : int
            The number of clusters to form as well as the number of centroids to generate.

        n_samples : int
            The number of samples to draw from each cluster.

        kmeans_save_path : str
            The path (including file name) where the resulting KMeans object should be saved.

        clusters_save_path : str
            The path (including file name) where the resulting clusters should be saved.

        ensure_correctness : bool, optional (default=False)
            If True, performs additional correctness checks, such as comparing SMILES string derived features to features in mols array.
            This requires 'path_to_pca' to be set.

        path_to_pca : str, optional (default=None)
            If ensure_correctness is True, this should be the path to a PCA model used to transform the molecules' descriptors.

        Returns
        -------
        cluster_to_samples : dict
            A dictionary where the keys are cluster labels and the values are lists of sampled SMILES strings from each cluster.

        Raises
        ------
        AssertionError
            If the number of requested samples exceeds the total number of molecules provided.
            If ensure_correctness is True but path_to_pca is None.
            If the number of labels returned by the KMeans algorithm differs from the number of molecules.
            If features calculated from a smile string differ from features in the mols array.
            If the total number of sampled molecules doesn't equal to n_clusters * n_samples.

    """
    assert n_clusters * n_samples <= len(mols), f"{n_clusters=} * {n_samples=} = {n_clusters*n_samples} requested but only {len(mols)} molecules provided"
    if ensure_correctness:
        assert path_to_pca is not None, "path_to_pca must be provided to ensure correctness"
        scaler, pca = pickle.load(open(path_to_pca, 'rb'))

    kmeans = _cluster_mols_experimental(mols=mols, n_iter=100, n_clusters=n_clusters, save_path=kmeans_save_path, objective='mixed', mixed_objective_loss_quantile=0.05)
    assert len(kmeans.labels_) == len(mols_smiles), "Number of labels differs from number of molecules"

    cluster_to_mols = {}
    for mol, label, smile in zip(mols, kmeans.labels_, mols_smiles):
        cluster_to_mols.setdefault(label, []).append(smile)
        if ensure_correctness: # recalculate descriptors from a smile string and compare to the descriptors in the array
            smile_features = pca.transform(scaler.transform(pd.DataFrame({k: [v] for k, v in rdkit.Chem.Descriptors.CalcMolDescriptors(rdkit.Chem.MolFromSmiles(smile)).items()})[scaler.get_feature_names_out()]))
            assert np.allclose(smile_features[0], mol), "Features calculated from a smile string differ from features in the array"

    # What happens below is sampling from each cluster. All the extra code is to ensure that the number of samples requested from each cluster
    # doesn't exceed the total number of available molecules. This is done by calculating the average number of molecules per cluster and then
    # calculating the number of extra molecules that need to be sampled from each cluster. The extra molecules are then distributed among the
    # clusters uniformly. If the number of extra molecules is greater than the number of molecules in a cluster, all
    # molecules from that cluster are sampled.
    avg_len = np.mean([len(v) for v in cluster_to_mols.values()])
    cluster_to_samples = {}
    extra_mols = 0
    left_to_sample = n_clusters*n_samples
    cluster_to_len = {cluster:len(mols) for cluster, mols in cluster_to_mols.items()}
    for i, (cluster, _) in enumerate(sorted(cluster_to_len.items(), key=lambda x: x[1], reverse=False)):
        smiles = cluster_to_mols[cluster]
    # for i, (cluster, smiles) in enumerate(cluster_to_mols.items()):
        if extra_mols > 0:
            cur_extra = int(1+extra_mols/(len(cluster_to_mols) - i) * len(smiles)/avg_len)
            cur_samples = n_samples + cur_extra
            extra_mols -= cur_extra
        else:
            cur_samples = n_samples
        if cur_samples > left_to_sample:
            cur_samples = left_to_sample

        if len(smiles) > cur_samples:
            cluster_to_samples[cluster] = random.sample(smiles, cur_samples)
            left_to_sample -= cur_samples
        else:
            cluster_to_samples[cluster] = smiles
            left_to_sample -= len(smiles)
            extra_mols += cur_samples - len(smiles)

    assert (n_sampled:=sum(len(vals) for vals in cluster_to_samples.values())) == n_clusters*n_samples, f"Sampled {n_sampled} but were requested {n_clusters*n_samples}"
    pickle.dump(cluster_to_mols, open(clusters_save_path, 'wb'))
    pickle.dump(cluster_to_samples, open(clusters_save_path.split('.')[0] + '_samples.pickle', 'wb'))
    return cluster_to_samples


cluster_and_sample(mols=pca_transformed, mols_smiles=gpt_smiles, n_clusters=100, n_samples=10,
                   kmeans_save_path=inference_config_dict['checkpoint_dir'] + 'k100means_07_07.pickle', ensure_correctness=False,
                   clusters_save_path=inference_config_dict['checkpoint_dir'] + 'cluster_to_samples_07_07.pickle')

#Docking pose generation

In [ ]:
import pickle
cluster_to_samples = pickle.load(open(inference_config_dict['checkpoint_dir'] + 'cluster_to_samples_07_07_samples.pickle', 'rb'))
print(cluster_to_samples)

In [ ]:
import pandas as pd
keyToData = {}
for cluster, mols in cluster_to_samples.items():
    for mol in mols:
        keyToData.setdefault('smiles', []).append(mol)
        keyToData.setdefault('cluster_id', []).append(cluster)
pd.DataFrame(keyToData).to_csv(inference_config_dict['checkpoint_dir']+'mols_sampled_for_difdock_07_07.csv')
pd.read_csv(inference_config_dict['checkpoint_dir']+'mols_sampled_for_difdock_07_07.csv').head()

##Set up notebook

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# install necessary packages
!pip install rdkit
!pip install pandas==1.5.3
!pip install molsets
!pip install wandb

!pip install pyg==0.7.1 --quiet
!pip install pyyaml==6.0 --quiet
!pip install scipy==1.7.3 --quiet
!pip install networkx==2.6.3 --quiet
!pip install biopython==1.79 --quiet
!pip install rdkit-pypi==2022.03.5 --quiet
!pip install e3nn==0.5.0 --quiet
!pip install spyrmsd==0.5.2 --quiet
!pip install pandas==1.5.3 --quiet
!pip install biopandas==0.4.1 --quiet

!pip install torch-geometric torch-scatter torch-sparse torch-cluster\
 -f https://data.pyg.org/whl/torch-{torch.__version__}.html

 # import necessary packages
from rdkit import Chem
import shutil
import torch
from torch import Tensor
import torch.nn as nn
from torch_geometric.nn.conv import MessagePassing, GatedGraphConv
from torch_geometric.nn import global_add_pool
from torch_geometric.utils import add_self_loops, dense_to_sparse
from torch_geometric.nn.aggr import AttentionalAggregation
from torch._C import NoneType
from torch.optim import Adam
from torch_geometric.data import Data, DataListLoader
from torch_geometric.nn import DataParallel as GeometricDataParallel
from torch.utils.data import Dataset
import numpy as np
import h5py
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, pairwise_distances
import os
import random
from scipy import stats
import matplotlib.pyplot as plt
import pkg_resources
pkg_resources.require("pandas==1.5.3")
import pandas as pd
import re
import math
import random
import logging
import wandb
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
import torch.optim as optim
from torch.cuda.amp import GradScaler
from rdkit import Chem
from rdkit.Chem import QED, Crippen
from rdkit.Contrib.SA_Score import sascorer
from rdkit.Chem.rdMolDescriptors import CalcTPSA
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
import moses
from moses.utils import get_mol

if not os.path.exists("/content/DiffDock"):
    %cd /content
    !git clone https://github.com/gcorso/DiffDock.git
    %cd /content/DiffDock
    !git checkout a6c5275 # remove/update for more up to date code

# clone ESM repository
if not os.path.exists("/content/DiffDock/esm"):
    %cd /content/DiffDock
    !git clone https://github.com/facebookresearch/esm
    %cd /content/DiffDock/esm
    !git checkout ca8a710
    !sudo pip install -e .
    %cd /content/DiffDock

# set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

##Run DiffDock to get top poses

In [ ]:
def get_top_poses(ligands_csv_path, protein_pdb_path):
    data = pd.read_csv(ligands_csv_path)
    ligand_files = []

    os.environ['HOME'] = 'esm/model_weights'
    os.environ['PYTHONPATH'] = f'{os.environ.get("PYTHONPATH", "")}:/content/DiffDock/esm'

    for i in range(len(data)):  # change 1 to len(data) for processing all ligands
        print(str((i / len(data)) * 100)[:5], ' %')
        smiles = data['smiles'][i]
        rdkit_mol = Chem.MolFromSmiles(smiles)

        if rdkit_mol is not None:
            with open('/content/input_protein_ligand.csv', 'w') as out:
                out.write('protein_path,ligand\n')
                out.write(f'{protein_pdb_path},{smiles}\n')

            # Clear out old results if running multiple times
            shutil.rmtree('/content/DiffDock/results', ignore_errors=True)

            # ESM Embedding Preparation
            os.chdir('/content/DiffDock')
            !python /content/DiffDock/datasets/esm_embedding_preparation.py --protein_ligand_csv /content/input_protein_ligand.csv --out_file /content/DiffDock/data/prepared_for_esm.fasta

            # ESM Extraction
            !python /content/DiffDock/esm/scripts/extract.py esm2_t33_650M_UR50D /content/DiffDock/data/prepared_for_esm.fasta /content/DiffDock/data/esm2_output --repr_layers 33 --include per_tok --truncation_seq_length 30000

            # Inference
            !python /content/DiffDock/inference.py --protein_ligand_csv /content/input_protein_ligand.csv --out_dir /content/DiffDock/results/user_predictions_small --inference_steps 20 --samples_per_complex 10 --batch_size 6

            # Move results
            for root, dirs, files in os.walk('/content/DiffDock/results/user_predictions_small'):
                for file in files:
                    if file.startswith('rank1_confidence'):
                        shutil.move(os.path.join(root, file), os.path.join('/content', 'drive', 'MyDrive', 'Generative_ML', 'diffdock_best_poses', f'complex{i}'))
                        ligand_files.append(f'/content/drive/MyDrive/Generative_ML/diffdock_best_poses/complex{i}')

    return ligand_files

In [ ]:
# get top DiffDock poses
top_diffdock_poses = get_top_poses('/content/drive/MyDrive/Generative_ML/data/molgpt_generated_nocond_06_10_fintetune2.csv',
                                   '/content/drive/MyDrive/Generative_ML/data/6o56.pdb')

In [ ]:
# write list of file paths to txt file
with open('/content/drive/MyDrive/Generative_ML/diffdock_poses.txt', 'w') as file:
    for path in top_diffdock_poses:
        file.write(path + '\n')

#Active learning based on pseudo protein-ligand binding energy

##Set up notebook

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
import condacolab

!mamba install pymol-open-source --yes

!pip install prolif
!pip install rdkit

from pymol import cmd
import prolif
from prolif.plotting.network import LigNetwork
from rdkit import Chem
from IPython.display import Image
import os
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

##Count connections

In [ ]:
interaction_scores = {
    'Hydrophobic': -2.5,
    'HBDonor': -3.5,
    'HBAcceptor': -3.5,
    'Anionic': -7.5,
    'Cationic': -7.5,
    'CationPi': -2.5,
    'PiCation': -2.5,
    'VdWContact': -1.0,
    'XBAcceptor': -3.0,
    'XBDonor': -3.0,
    'FaceToFace': -3.0,
    'EdgeToFace': -1.0,
    'MetalDonor': -3.0,
    'MetalAcceptor': -3.0,
}

def get_contacts(protein, ligand):
  cmd.delete('all')
  cmd.load(protein, 'protein')
  cmd.h_add('protein')
  cmd.remove('sol')
  cmd.save('/content/protein.pdb')

  prot = prolif.Molecule(Chem.MolFromPDBFile('/content/protein.pdb', removeHs=False))
  lig = Chem.SDMolSupplier(ligand, removeHs=False)
  lig = prolif.Molecule.from_rdkit(lig[0])

  fp = prolif.Fingerprint(interactions=list(interaction_scores.keys()))
  fp.run_from_iterable([lig], prot, progress=False)
  try:
    df = fp.to_dataframe(return_atoms=True)
    df_stacked = df.stack(level=[0, 1, 2])
    df_reset = df_stacked.to_frame().reset_index()
    df_reset.columns = ['Frame', 'ligand', 'protein', 'interaction', 'value']
    df_reset['score'] = df_reset['interaction'].apply(lambda x: interaction_scores[x])
    return df_reset['score'].sum()
  except:
    print('Complex has no meaningful protein-ligand connections')
    return int(0)

In [ ]:
import os
ligand_list = []
for i in os.listdir('/content/drive/MyDrive/Generative_ML/best_poses/'):
    comp_num = str(i)
    for j in os.listdir('/content/drive/MyDrive/Generative_ML/best_poses/' + comp_num):
        if j.startswith('index0'):
            ligand_list.append('/content/drive/MyDrive/Generative_ML/best_poses/' + comp_num + '/' + j + '/' + 'rank1.sdf')

In [ ]:
from pymol import cmd
import prolif
from rdkit import Chem
count = 0
good_ligands = {}
for i in ligand_list:
    if i.split('/')[6].split('.')[0] not in good_ligands:
        score = get_contacts('/content/drive/MyDrive/Generative_ML/data/HNH.pdb', i)
        good_ligands[i.split('/')[6].split('.')[0]] = score
        count += 1
        if int(count%(len(ligand_list)/100)) == 0:
            print(int(count/len(ligand_list)*100), ' %')

In [ ]:
sorted_ligands = {k: v for k, v in sorted(good_ligands.items(), key=lambda item: item[1])}

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np

data = list(sorted_ligands.values())

plt.figure(figsize=(8, 6), dpi=80)

plt.hist(data, bins=50, density=True, color='gray', alpha=0.7, edgecolor='black')

smoothed_data = np.linspace(min(data), max(data), 1000)
kde = gaussian_kde(data)
smoothed_line = kde(smoothed_data)

plt.plot(smoothed_data, smoothed_line, linewidth=2.5, color='black')

plt.xlabel("Values", fontsize=18)
plt.ylabel("Density", fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.title("Density Plot of Scores", fontsize=20)

plt.grid(linestyle='--', linewidth=0.5, alpha=0.7)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.tick_params(axis='both', which='both', direction='in', length=4)

plt.show()

print(f'There are {list(sorted_ligands.values()).count(0)} ligands with 0 connections ({np.round(list(sorted_ligands.values()).count(0) / len(sorted_ligands)*100, 1)}%)')
print(f'There are {len([value for value in list(sorted_ligands.values()) if value < -10.0])} ligands with scores less than -10.0 ({np.round(len([value for value in list(sorted_ligands.values()) if value < -10.0]) / len(sorted_ligands)*100, 1)}%)')
print(f'The mean score for all ligands is {np.round(np.mean(list(sorted_ligands.values())), 1)}')
print(f'The lowest score for all ligands is {np.min(list(sorted_ligands.values()))}')

In [ ]:
good_ligands = {key: value for key, value in sorted_ligands.items() if value != 0}

In [ ]:
good_idx = [k[7:] for k in good_ligands if k.startswith('complex')]

In [ ]:
import pandas as pd
diffdock_input = pd.read_csv('/content/drive/MyDrive/Generative_ML/current_data/mols_sampled_for_difdock_07_07.csv')

In [ ]:
good_data = diffdock_input.iloc[good_idx]

In [ ]:
good_data['score'] = [good_ligands.get(f'complex{complex_number}', 0) for complex_number in good_data.index]

In [ ]:
good_data

## Prepare training dataset for next round of active learning

In [ ]:
cluster_to_scores = {}
for index, row in good_data.iterrows():
    cluster_to_scores.setdefault(row['cluster_id'], []).append(row['score'])
cluster_to_score = {cluster_id: np.mean(scores) for cluster_id, scores in cluster_to_scores.items()}
import random
cluster_to_score = {k: random.uniform(-20, 0) for k in range(50)}
print(cluster_to_score)

Here, we need to map the good data back to the original clusters and create a training set for active learning. THen, perform active learning, generate molecules, and repeat process. Finally, generate 2 million molecules to train BERT

In [ ]:
import pickle

def _preprocess_scores_uniformly(scores, remove_positives=False, lowest_score=1):
    """
        Preprocesses a dictionary of scores by negating and normalizing them.

        The function negates all scores and optionally removes positive scores. If the minimum value among the negated scores
        is less than zero, it shifts all values by subtracting the minimum value and adding 'lowest_score'. The final step is
        to normalize the scores so that their total sum equals to 1.

        Parameters
        ----------
        scores : dict
            A dictionary of scores where the keys are identifiers and the values are their corresponding scores.

        remove_positives : bool, optional (default=False)
            If True, all positive scores are removed after negation.

        lowest_score : int, optional (default=1)
            This value is added to all scores if the minimum score is less than zero.

        Returns
        -------
        normalized : dict
            The normalized dictionary of scores.

    """
    negated = {k: -v for k, v in scores.items()}
    min_value = min(negated.values())
    if min_value < 0:
        if remove_positives:
            negated = {k: v for k, v in negated.items() if v > 0}
        else:
            negated = {k: v - min_value + lowest_score for k, v in negated.items()}
    total = sum(negated.values())
    normalized = {k: v / total for k, v in negated.items()}
    return normalized

def _preprocess_scores_softmax(scores):
    negated = {k: -v for k, v in scores.items()}
    max_value = max(negated.values())
    exponentiate = {k: np.exp(v - max_value) for k, v in negated.items()}
    total = sum(exponentiate.values())
    softmax = {k: v / total for k, v in exponentiate.items()}
    return softmax

def balance_cluster_to_n(cluster_to_n, cluster_to_len):
    """
        Balances the target number of samples for each cluster to ensure it doesn't exceed the actual size of the cluster.

        The function first calculates the surplus (i.e., the excess of the target number over the actual size) for each cluster.
        Then, it distributes the total surplus proportionally among the clusters that have a deficit (i.e., the target number is less than the actual size).
        If after this distribution, there's still a deficit (i.e., the sum of target numbers is less than the sum of actual sizes), the function
        increases the target number of the largest clusters one by one until the sum of target numbers equals to the sum of actual sizes.

        Parameters
        ----------
        cluster_to_n : dict
            A dictionary mapping cluster identifiers to their target number of samples.

        cluster_to_len : dict
            A dictionary mapping cluster identifiers to the actual size of each cluster.

        Returns
        -------
        balanced : dict
            A dictionary mapping cluster identifiers to their balanced target number of samples.

        Raises
        ------
        AssertionError
            If the sum of target numbers before and after balancing don't match.

    """

    surplus = {key: cluster_to_n[key] - cluster_to_len[key] for key in cluster_to_n if cluster_to_n[key] > cluster_to_len[key]}
    balanced = {k:v for k, v in cluster_to_n.items()}
    n_to_cluster = {v: k for k, v in cluster_to_n.items()}

    for key in surplus:
        balanced[key] = cluster_to_len[key]

    total_surplus = sum(surplus.values())
    initial_n_sum = sum(n for key, n in cluster_to_n.items() if key not in surplus)

    for key in balanced:
        if key in surplus: continue
        surplus_to_add = total_surplus * cluster_to_n[key] / initial_n_sum
        new_n = int(cluster_to_n[key] + surplus_to_add)
        balanced[key] = min(new_n, cluster_to_len[key])

    deficit = sum(cluster_to_n.values()) - sum(balanced.values())
    while deficit > 0:
        for initial_n in sorted(n_to_cluster, reverse=True):
            if deficit == 0:
                break
            if (cluster:=n_to_cluster[initial_n]) in surplus: continue
            if balanced[cluster] < cluster_to_len[cluster]:
                balanced[cluster] += 1
                deficit -= 1

    assert sum(cluster_to_n.values()) == sum(balanced.values()), f"Before balancing had {sum(cluster_to_n.values())}, post balancing = {sum(balanced.values())}"
    return balanced

def sample_clusters_for_active_learning(cluster_to_scores, n_samples, path_to_clusters, probability_type='softmax', remove_positives=False, lowest_score=1):
    """
        Sample molecules from clusters for active learning purposes, considering previously docked molecules and balancing the sampling among clusters.

        This function uses either softmax or uniform probabilities to determine how many molecules to sample from each cluster. The function then samples
        the required number of new molecules (i.e., those not present in docked_mols) from each cluster. The sampling is balanced to ensure the target number
        doesn't exceed the actual size of the cluster.

        Parameters
        ----------
        cluster_to_scores : dict
            A dictionary mapping cluster identifiers to their scores.

        n_samples : int
            The total number of molecules to sample.

        path_to_clusters : str
            The path to a pickle file storing a dictionary that maps each cluster to a list of molecules.

        probability_type : str, optional (default='softmax')
            The type of probability distribution used to determine the number of samples per cluster.
            Options are 'softmax' and 'uniform'.

        remove_positives : bool, optional (default=False)
            Only used when probability_type is 'uniform'. If True, positive scores are removed after negation.

        lowest_score : int, optional (default=1)
            Only used when probability_type is 'uniform'. This value is added to all scores if the minimum score is less than zero.

        Returns
        -------
        training : list
            A list of randomly sampled molecules for active learning.

        Raises
        ------
        KeyError
            If an unsupported probability_type is provided.
        AssertionError
            If the number of sampled molecules doesn't equal to n_samples.

    """
    if probability_type == 'softmax':
        probability_function = _preprocess_scores_softmax
    elif probability_type == 'uniform':
        probability_function = lambda x: _preprocess_scores_uniformly(x, remove_positives, lowest_score)
    else:
        raise KeyError("Only uniform and softmax probabilities are supported")
    cluster_to_mols = pickle.load(open(path_to_clusters, 'rb'))
    cluster_to_samples = pickle.load(open(path_to_clusters.split('.')[0] + '_samples.pickle', 'rb'))
    docked_mols = {smile for smiles in cluster_to_samples.values() for smile in smiles}
    cluster_to_new_mols = {k: [smile for smile in v if smile not in docked_mols] for k, v in cluster_to_mols.items()}

    probabilities = probability_function(cluster_to_scores)
    cluster_to_n = {k: int(v * n_samples) for k, v in probabilities.items()}
    max_cluster_id, max_prob = None, 0
    for cluster, prob in probabilities.items():
        if prob > max_prob:
            max_cluster_id, max_prob = cluster, prob
    cluster_to_n[max_cluster_id] += n_samples - sum(cluster_to_n.values())

    cluster_to_len = {k: len(v) for k, v in cluster_to_new_mols.items()}
    balanced = balance_cluster_to_n(cluster_to_n, cluster_to_len)

    training = []
    for i, (cluster, n) in enumerate(balanced.items()):
        training.extend(random.sample(cluster_to_new_mols[cluster], n))
    assert len(training) == n_samples, f"{len(training)=} != {n_samples=}"
    return training

training_sampled = sample_clusters_for_active_learning(cluster_to_score, n_samples=1000, probability_type='uniform', path_to_clusters= '/content/drive/MyDrive/Generative_ML/current_data/cluster_to_samples_07_07.pickle')


In [ ]:
def combine_sampled_and_good_ligands(sampled, good_ligands, good_ligand_multiplier:int, save_path):
    # assert isinstance(good_ligand_multiplier, int), "A multiplier should be an integer"
    keyToData = {}
    for mol in sampled:
        keyToData.setdefault('smiles', []).append(mol)
    for mol in good_ligands:
        for _ in range(good_ligand_multiplier):
            keyToData.setdefault('smiles', []).append(mol)
    pd.DataFrame(keyToData).to_csv(save_path)


combine_sampled_and_good_ligands(training_sampled, good_ligands, good_ligand_multiplier=2, save_path='/content/drive/MyDrive/Generative_ML/current_data/active_learning_training.csv')
pd.read_csv('/content/drive/MyDrive/Generative_ML/current_data/active_learning_training.csv').shape

Now train and generate

#BERT

##Set up notebook

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# install necessary packages
!pip install rdkit
!pip install pandas==1.5.3
!pip install molsets
!pip install wandb

# clone Sophia optimizer GitHub repository
!git clone https://github.com/Liuhong99/Sophia.git

# import necessary packages
import numpy as np
import h5py
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, pairwise_distances
import os
from scipy import stats
import matplotlib.pyplot as plt
import pkg_resources
pkg_resources.require("pandas==1.5.3")
import pandas as pd
import re
import math
import random
import logging
import wandb
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
import torch.optim as optim
from torch.cuda.amp import GradScaler
from rdkit import Chem
from rdkit.Chem import QED, Crippen
from rdkit.Contrib.SA_Score import sascorer
from rdkit.Chem.rdMolDescriptors import CalcTPSA
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.DataStructs.cDataStructs import TanimotoSimilarity
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles
import moses
from moses.utils import get_mol
from Sophia.sophia import SophiaG
import yaml

# set random seed for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

##Utils & Dataset

In [ ]:
@torch.no_grad()
def sample(model, x, steps, temperature=1.0, prop=None, scaffold=None):
    block_size = model.get_block_size() # define size of context window used for input conditioning
    model.eval()
    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # limit conditioning input to the most recent block_size elements
        logits, _= model(x_cond, prop = prop, scaffold = scaffold) # give input to model and get logits (unnormalized scores or probabilities)
        logits = logits[:, -1, :] / temperature # extract the logits for the next token in the sequence
        probs = F.softmax(logits, dim=-1)
        ix = torch.multinomial(probs, 1)
        x = torch.cat((x, ix), dim=1) # concatenate the chosen token index with the existing sequence
    return x



@torch.no_grad()
def fill_mask(model, x, temperature=1.0, prop=None, scaffold=None):
    block_size = model.get_block_size()  # Get model's maximum context length
    model.eval()

    # Check if mask token is present
    mask_index = model.config.mask_index  # Assuming 'mask_index' is available in your model's config
    while mask_index in x[0]:  # If mask token is present
        # Find all mask positions
        mask_positions = (x[0] == mask_index).nonzero(as_tuple=True)

        for pos in mask_positions:  # Iterate over each mask position
            # Limit conditioning input to the most recent block_size elements
            x_cond = x if x.size(1) <= block_size else x[:, -block_size:]
            # Get token probabilities
            logits, _ = model(x_cond, prop=prop, scaffold=scaffold)

            # Extract the logits for the masked token position, apply temperature and softmax to get probabilities
            logits = logits[0, pos, :] / temperature
            probs = F.softmax(logits, dim=-1)
            ix = torch.multinomial(probs, 1)  # Sample token from probability distribution

            x[0, pos] = ix  # Replace mask token with sampled token

    return x  # Return unmasked tensor

def check_novelty(gen_smiles, train_smiles):
    if len(gen_smiles) == 0:
        novel_ratio = 0
    else:
        duplicates = [1 for mol in gen_smiles if mol in train_smiles]
        novel = len(gen_smiles) - sum(duplicates)
        novel_ratio = novel*100/len(gen_smiles)
    return novel_ratio

def canonic_smiles(smiles_or_mol):
    mol = get_mol(smiles_or_mol)
    if mol is None:
        return None
    return Chem.MolToSmiles(mol)

class SMILESDataset(Dataset):

    def __init__(self, data=None, content=None, block_size=None, prop=None, scaffold=None, scaffold_maxlen=None, len_data=None, mask_prob=0.15):
        if content is None:
            self.desc_only = True
            return
        self.desc_only = False
        self.chars = sorted(list(set(content)))
        data_size, vocab_size = len(content), len(self.chars)
        self.vocab_size = vocab_size

        self.stoi = {ch:i for i,ch in enumerate(self.chars)}
        self.itos = {i:ch for i,ch in enumerate(self.chars)}
        self.max_len = block_size
        self.data = data
        self.prop = prop
        self.sca = scaffold
        self.scaf_max_len = scaffold_maxlen
        self.len_data = len_data
        self.mask_prob = mask_prob

        # add X token to vocabulary
        self.stoi['X'] = len(self.stoi)
        self.itos[len(self.itos)] = 'X'
        self.vocab_size = len(self.stoi)

    def export_desc_attributes(self, export_path):
        attr_dict = {
            "desc_only": self.desc_only,
            "vocab_size": self.vocab_size,
            "max_len": self.max_len,
            "stoi": self.stoi,
            "itos": self.itos,
            "scaf_max_len": self.scaf_max_len,
            "len_data": self.len_data
        }
        with open(export_path, 'w') as f:
            yaml.dump(attr_dict, f)

    def load_desc_attributes(self, load_path):
        with open(load_path, 'r') as f:
            attr_dict = yaml.load(f, Loader=yaml.SafeLoader)
        self.__dict__.update(attr_dict)

    def __len__(self):
        assert not self.desc_only, "Dataset is not initialized"
        return len(self.data)

    def __getitem__(self, idx):
        assert not self.desc_only, "Dataset is not initialized"
        smiles, prop, scaffold = self.data[idx].strip(), self.prop[idx], self.sca[idx]
        if scaffold:
          scaffold=scaffold.strip()
        pattern =  "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9]|X)"
        regex = re.compile(pattern)

        #tokenize input string
        tokens_arr=regex.findall(smiles)

        #Save ground truth encodings before token masking
        smiles_true = smiles+str('<')*(self.max_len - len(tokens_arr))
        if len(tokens_arr) > self.max_len:
            smiles_true = smiles[:self.max_len]
        dix_true=[self.stoi[s] for s in regex.findall(smiles_true)]



        # randomly replace tokens
        mask_idx = [] #True indicates that the corresponding position will be ignored for computing loss
        for s in range(len(tokens_arr)):
          if random.random() < .15:
            mask_idx.append(False)
            num = random.random()
            if num >= .2: # 80%
                tokens_arr[s]='X'
            elif num >= 0.1: # 10%
                tokens_arr[s] = self.chars[int(random.random()*len(self.chars))]
          else:
            mask_idx.append(True)

        #pad the mask appropriately
        while len(mask_idx)<self.max_len:
            mask_idx.append(True)

        assert len(mask_idx)==self.max_len
        smiles=''.join(tokens_arr)

        smiles += str('<')*(self.max_len - len(regex.findall(smiles)))
        if len(regex.findall(smiles)) > self.max_len:
            smiles = smiles[:self.max_len]

        assert len(''.join(regex.findall(smiles)))==len(smiles)
        smiles=regex.findall(smiles)


        dix =  [self.stoi[s] for s in smiles]
        if scaffold:
          scaffold += str('<')*(self.scaf_max_len - len(regex.findall(scaffold)))
          if len(regex.findall(scaffold)) > self.scaf_max_len:
              scaffold = scaffold[:self.scaf_max_len]
          scaffold=regex.findall(scaffold)
          sca_dix = [self.stoi[s] for s in scaffold]
          sca_tensor = torch.tensor(sca_dix, dtype=torch.long)
        else:
          sca_tensor=torch.tensor(scaffold,dtype=torch.bool)
        x = torch.tensor(dix, dtype=torch.long)
        y = torch.tensor(dix_true, dtype=torch.long)
        prop = torch.tensor([prop], dtype=torch.float)
        return x, y, prop, sca_tensor, torch.tensor(mask_idx)

##Model

In [ ]:
class GPTConfig:
    def __init__(self, vocab_size=None, block_size=None, mask_index=None, **kwargs):
        self.vocab_size = vocab_size
        self.block_size = block_size
        self.mask_index = mask_index
        for k,v in kwargs.items():
            setattr(self, k, v)

    def export_attributes(self, export_path):
        with open(export_path, 'w') as f:
            yaml.dump(vars(self), f)

    def load_attributes(self, load_path):
        with open(load_path, 'r') as f:
            config_dict = yaml.load(f, Loader=yaml.SafeLoader)
        self.__dict__.update(config_dict)

class SelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embed % config.n_head == 0
        self.config = config

        self.query = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)
        self.key = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)
        self.value = nn.Linear(config.n_embed, config.n_embed, bias=config.att_bias)

        self.attn_drop = nn.Dropout(config.att_drop_rate)
        self.resid_drop = nn.Dropout(config.att_drop_rate)

        self.proj = nn.Linear(config.n_embed, config.n_embed)
        self.n_head = config.n_head

        num = int(bool(config.num_props)) + int(config.scaffold_maxlen)
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size + num, config.block_size + num))
                                .view(1, 1, config.block_size + num, config.block_size + num))

    def forward(self, x, layer_past=None):
        B, T, C = x.size()
        # apply attention functions to get tensors with dimensions (B, n_head, T, head_size)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head)
        k = self.key(x).view(B, T, self.n_head, C // self.n_head)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head)
        if self.config.do_flash:
            q = q.transpose(1, 2)
            k = k.transpose(1, 2)
            v = v.transpose(1, 2)
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, dropout_p=self.config.att_drop_rate if self.training else 0, is_causal=self.config.is_causal)
            y = y.transpose(1, 2)
        else:
            # (B h T s) @ (B h s T) -> (B h T T)
            att = torch.einsum('bths,bihs->bhti', q, k) / math.sqrt(k.size(-1))
            att = att.masked_fill(self.mask[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            # (B h T T) @ (B h T s) -> (B h T s)
            y = torch.einsum('bhtq,bqhs->bths', att, v)
            self.att_weights = att
        self.attended = y
        y = y.contiguous().view(B, T, C)
        y = self.resid_drop(self.proj(y))
        self.out = y
        return y

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embed)
        self.ln2 = nn.LayerNorm(config.n_embed)
        self.attn = SelfAttention(config)
        self.mlp = nn.Sequential(nn.Linear(config.n_embed, config.ff_mult*config.n_embed), nn.GELU() if config.doGELU else nn.ReLU(),
            nn.Linear(config.ff_mult*config.n_embed, config.n_embed), nn.Dropout(config.att_drop_rate))

    def forward(self, x):
        y = self.attn(self.ln1(x))
        x = x + y # perform a residual connection by summing input and attention output
        x = x + self.mlp(self.ln2(x)) # apply layer normalization and then MLP, create a residual connection with input
        return x

class GPT(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embed)
        self.type_emb = nn.Embedding(2, config.n_embed)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embed))
        # if conditioning on at least 1 property:
        if config.num_props:
            # initialize property linear layer, map property vector to embedding dimension
            self.prop_nn = nn.Linear(config.num_props, config.n_embed)

        self.drop = nn.Dropout(config.gpt_drop_rate)
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])

        self.ln_f = nn.LayerNorm(config.n_embed)
        self.head = nn.Linear(config.n_embed, config.vocab_size, bias=config.gpt_bias)
        self.block_size = config.block_size # define the context size
        self.apply(self._init_weights) # initialize weights and apply to all relevant modules in the model

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        decay, no_decay = set(), set()
        no_decay = set()

        whitelist_weight_modules = (torch.nn.Linear)
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        # for named module of the model:
        for mn, m in self.named_modules():
            # for named parameter of each module:
            for pn, p in m.named_parameters():
                # construct full parameter name by concatenating module name and parameter name, separated by a dot
                fpn = '%s.%s' % (mn, pn) if mn else pn
                if pn.endswith('bias') or ('bias' in pn):
                    no_decay.add(fpn)
                elif (pn.endswith('weight') or ('weight' in pn)) and isinstance(m, whitelist_weight_modules):
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    no_decay.add(fpn)
        no_decay.add('pos_emb')
        param_dict = {pn:p for pn, p in self.named_parameters()}
        assert len(decay & no_decay) == 0
        # assert that all parameters from both sets have been correctly separated
        assert len(param_dict.keys() - (decay | no_decay)) == 0
        optim_groups = [{"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
                        {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0}]
        optimizer = SophiaG(optim_groups, lr=train_config.learning_rate, betas=train_config.betas, rho=train_config.rho, weight_decay=train_config.weight_decay)
        return optimizer

    def forward(self, idx, targets=None, prop=None, scaffold=None):
        b, t = idx.size()
        assert t <= self.block_size
        if self.config.num_props:
            assert prop.size(-1) == self.config.num_props, f"number of properties {prop.size(-1)=} doesn't match the expected size {self.config.num_props=}"
        token_embeddings = self.tok_emb(idx)
        position_embeddings = self.pos_emb[:, :t, :]
        type_embeddings = self.type_emb(torch.ones((b,t), dtype=torch.long, device=idx.device))
        x = self.drop(token_embeddings + position_embeddings + type_embeddings)
        # Condition on properties
        if self.config.num_props:
            type_embd = self.type_emb(torch.zeros((b, 1), dtype=torch.long, device=idx.device))
            if prop.ndim == 2:
                p = self.prop_nn(prop.unsqueeze(1))
            else:
                p = self.prop_nn(prop)
            p += type_embd
            x = torch.cat([p, x], 1)
        # Condition on scaffold
        if self.config.scaffold:
            type_embd = self.type_emb(torch.zeros((b, 1), dtype = torch.long, device = idx.device))
            scaffold_embeds = self.tok_emb(scaffold)
            scaffold_embeds += type_embd
            x = torch.cat([scaffold_embeds, x], 1)

        # Transformer blocks
        for layer in self.blocks:
            x = layer(x)
        x = self.ln_f(x)
        logits = self.head(x)

        if self.config.num_props and self.config.scaffold:
            num = int(bool(self.config.num_props)) + int(self.config.scaffold_maxlen)
        elif self.config.num_props:
            num = int(bool(self.config.num_props))
        elif self.config.scaffold:
            num = int(self.config.scaffold_maxlen)
        else:
            num = 0
        # Slice the logits tensor along the second dimension to exclude the first num elements
        logits = logits[:, num:, :]

        return logits

##Training

In [ ]:
class TrainerConfig:
    epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.965, 0.99) #(0.9, 0.95)
    rho = 0.04 # For SophiaG
    weight_decay = 0.1

    lr_decay = False
    warmup_tokens = 375e6 # number of warm-up tokens for learning rate decay
    final_tokens = 260e9 # number of tokens at which the learning rate decays to 10% of the original
    num_workers = 0 # number of worker processes to use for loading data

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

def loss_function(logits, y, mask_idx):
    loss=nn.CrossEntropyLoss(ignore_index=-1)
    y[mask_idx]=-1
    return loss(logits.view(-1,logits.size(-1)),y.view(-1))

class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config, stoi, itos):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config
        self.stoi = stoi
        self.itos = itos
        self.model = self.model.to(config.device)

    def train(self, wandb):
        model, config = self.model, self.config
        optimizer = model.configure_optimizers(config)
        scaler = GradScaler() # define variable used for gradient scaling in mixed-precision training
        self.tokens = 0 # initialize a counter used for learning rate decay

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True, batch_size=config.batch_size, num_workers=config.num_workers)
            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            # for batch index, batch in progress bar:
            for it, (x, y, p, scaffold, mask_idx) in pbar:
                # move the input data tensor, target data tensor, property tensor, and scaffold tensor to GPU
                x, y, p, scaffold , mask_idx = x.to(config.device), y.to(config.device), p.to(config.device), scaffold.to(config.device), mask_idx.to(config.device)
                # allow model to use lower-precision computations for improved memory usage
                if config.device == 'cuda':
                    with torch.cuda.amp.autocast():
                        with torch.set_grad_enabled(is_train):
                            logits= model(x, y, p, scaffold)
                            loss = loss_function(logits, y, mask_idx)
                            loss = loss.mean()
                            losses.append(loss.item())
                else:
                    with torch.cpu.amp.autocast():
                        with torch.set_grad_enabled(is_train):
                            logits = model(x, y, p, scaffold)
                            loss = loss_function(x, y, mask_idx)
                            loss = loss.mean()
                            losses.append(loss.item())

                if is_train:
                    model.zero_grad()
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer) # unscale the gradients of the optimizer's parameters to their original values
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients of model parameters to prevent them from exploding, setting maximum gradient norm to be 1.0
                    scaler.step(optimizer) # update the optimizer's parameters based on calculated gradients
                    scaler.update() # update the scale factor of the gradient scaler
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # increment the number of processed tokens by the count of valid tokens (not padding or special tokens)
                        if self.tokens < config.warmup_tokens:
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens)) # perform a linear warm-up
                        else:
                            # calculate the progress of training in terms of the number of tokens processed
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            # calculate the scaling factor for the learning rate (between 0.1 and 1.0)
                            # to gradually reduce learning rate as training progresses
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult # multiply the base learning rate by the scaling factor to obtain the updated learning rate
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate
                    # log training progress using Weights & Biases
                    if wandb is not None:
                        wandb.log({'step_train_loss': loss, 'train_step': it + epoch*len(loader), 'learning_rate': lr})
                    # update the description of the progress bar with epoch, iteration, and training loss
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
            return float(np.mean(losses))


        # initialize best loss as infinity
        best_loss = float('inf')
        for epoch in range(config.epochs):
            print(f'{epoch=}')
            train_loss = run_epoch('train')
            if self.test_dataset is not None:
                test_loss = run_epoch('test')
            if wandb is not None:
                wandb.log({'epoch_valid_loss': test_loss, 'epoch_train_loss': train_loss, 'epoch': epoch + 1})
            good_model = self.test_dataset is None or test_loss < best_loss
            if good_model:
                best_loss = test_loss
                torch.save(self.model.state_dict(), self.config.ckpt_path)

In [ ]:
def load_data(train_config_dict):
    if (cut:=train_config_dict["slice_data"]):
        train_data = pd.read_csv(train_config_dict["train_path"])[:cut]
        val_data = pd.read_csv(train_config_dict["val_path"])[:cut]
    else:
        train_data = pd.read_csv(train_config_dict["train_path"])
        val_data = pd.read_csv(train_config_dict["val_path"])

    smiles = train_data['smiles']
    vsmiles = val_data['smiles']

    prop = train_data[train_config_dict["props"]].values.tolist()
    vprop = val_data[train_config_dict["props"]].values.tolist()

    scaffold = train_data['scaffold_smiles']
    vscaffold = val_data['scaffold_smiles']

    # define a regular expression that matches molecular tokens in SMILES strings
    pattern = "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9]|X)"
    # compile pattern into a regular expression object that can be used for matching operations
    regex = re.compile(pattern)

    context = {'<'}

    max_len, scaffold_max_len = 0, 0
    for iterator in (smiles.values, vsmiles.values):
        for i in iterator:
            chars = regex.findall(i.strip())
            max_len = max(max_len, len(chars))
            for char in chars:
                context.add(char)
    for iterator in (scaffold.values, vscaffold.values):
        for i in iterator:
            chars = regex.findall(i.strip())
            scaffold_max_len = max(scaffold_max_len, len(chars))
            for char in chars:
                context.add(char)
    print(max_len)
    context = sorted(list(context))

    smiles = [i + str('<')*(max_len - len(regex.findall(i.strip()))) for i in smiles]
    vsmiles = [i + str('<')*(max_len - len(regex.findall(i.strip()))) for i in vsmiles]
    scaffold = [i + str('<')*(scaffold_max_len - len(regex.findall(i.strip()))) for i in scaffold]
    vscaffold = [i + str('<')*(scaffold_max_len - len(regex.findall(i.strip()))) for i in vscaffold]

    # if not conditioning on scaffolds: define 'scaffold' as a list of length SMILES string filled with 'False' values
    scaffold=[False]*len(smiles) if not train_config_dict["use_scaf"] else scaffold
    train_dataset = SMILESDataset(smiles, context, max_len, prop=prop, scaffold=scaffold, scaffold_maxlen=scaffold_max_len, len_data=len(train_data), mask_prob=0.15)
    valid_dataset = SMILESDataset(vsmiles, context, max_len, prop=vprop, scaffold=vscaffold, scaffold_maxlen=scaffold_max_len, len_data=len(val_data), mask_prob=0.15)
    train_dataset.export_desc_attributes(train_config_dict["desc_path"])
    return train_dataset, valid_dataset

In [ ]:
def pretrain_BERT(train_dataset, valid_dataset, model_config_dict, train_config_dict):
  """
  OUTPUTS:
  1) checkpoint of trained model parameters
  2) Weights & Biases logged run
  """

  mask_index = train_dataset.stoi['X']
  mconf = GPTConfig(train_dataset.vocab_size, train_dataset.max_len, mask_index=mask_index, num_props=len(train_config_dict["props"]), scaffold=train_config_dict["use_scaf"], scaffold_maxlen=train_dataset.scaf_max_len, **model_config_dict)
  model = GPT(mconf).to(model_config_dict["device"])

  if train_config_dict['load_cpt'] != None:
    model.load_state_dict(torch.load(train_config_dict['load_cpt']))
  torch.compile(model)

  tconf = TrainerConfig(warmup_tokens=0.1*train_dataset.len_data*train_dataset.max_len, final_tokens=train_config_dict["epochs"]*train_dataset.len_data*train_dataset.max_len, block_size=train_dataset.max_len, **train_config_dict)
  trainer = Trainer(model, train_dataset, valid_dataset, tconf, train_dataset.stoi, train_dataset.itos)

  %env WANDB_EXECUTABLE=python3
  wandb.init(project="mol_transformer", name=train_config_dict["wandb_runname"])
  trainer.train(wandb=wandb)
  return model, tconf

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
BASE = '/content/drive/MyDrive/Generative_ML/'

model_config_dict = {
    "device": DEVICE,
    "att_bias": False,
    "gpt_bias": True,
    "att_drop_rate": 0.1,
    "gpt_drop_rate": 0.1,
    "n_layer": 8,
    "n_head": 8,
    "n_embed": 256,
    "ff_mult": 4, # multiplier for FF inside multihead,
    "doGELU": True, # else ReLU
    "attention_times": [],
    "do_flash": True,
    "is_causal": False
}

train_config_dict = {
    "desc_path": BASE + 'checkpoints/descriptors_10k.yaml',
    "train_path": BASE + 'data/MOSES_processed_train.csv',
    "val_path": BASE + 'data/MOSES_processed_val.csv',
    "slice_data": False,
    "ckpt_path": BASE + 'checkpoints/7-03_all_mask.pt',
    "wandb_runname": "7_03_all_mask",
    "use_scaf": False,
    "props": [],
    "device": DEVICE,
    "epochs": 10,
    "batch_size": 512,
    "lr_decay": True,
    "num_workers": 0,
    "load_cpt": None
}

In [ ]:
train_dataset, val_dataset = load_data(train_config_dict)

In [ ]:
# define function to train BERT model
model, tconf = pretrain_BERT(
                train_dataset = train_dataset,
                valid_dataset = val_dataset,
                model_config_dict = model_config_dict,
                train_config_dict = train_config_dict
        )

#GK wandb API Key: c99c9a01523f93287716691fa3360b1f4566e115
#RB wandb API Key: 4d3d628c6b5a4b3554c7a89ea50df8a4a6be0f85

##Generation & Evaluation

In [ ]:
def generate_SMILES(model_config_dict, inference_config_dict):
  props = inference_config_dict["props"]
  scaffold = inference_config_dict["scaffold"]

  # define a regular expression that matches molecular tokens in SMILES strings
  pattern = "(\[[^\]]+]|<|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9]|X)"
  regex = re.compile(pattern)

  dataset = SMILESDataset()
  dataset.load_desc_attributes(inference_config_dict['desc_path'])
  use_scaf = False if scaffold is None else True

  mconf = GPTConfig(dataset.vocab_size, dataset.max_len, num_props=len(props), scaffold=use_scaf, scaffold_maxlen=dataset.scaf_max_len, **model_config_dict)
  model = GPT(mconf).to(model_config_dict['device'])
  torch.compile(model)

  # load parameters into the model
  model.load_state_dict(torch.load(inference_config_dict["model_params"], map_location=torch.device(model_config_dict['device'])))
  block_size = model.get_block_size() #inference_config_dict["block_size"]
  assert block_size == dataset.max_len, "Warning: model block size and dataset block size are different"
  # calculate number of generation iterations from total number of SMILES to generate and batch size
  gen_iter = math.ceil(inference_config_dict["gen_size"] / inference_config_dict["batch_size"])
  stoi = dataset.stoi # define dictionary to map strings to integers
  itos = dataset.itos # define dictionary to map integers to strings
  # is a scaffold is defined for conditioning:
  if scaffold is not None:
      # pad '<' to end of scaffold string to achieve maximum scaffold length
      scaffold += str('<')*(dataset.scaf_max_len - len(regex.findall(scaffold)))
      # convert the scaffold SMILES to a tensor of integers and repeat along the batch dimension, move to GPU
      scaffold=torch.tensor([stoi[s] for s in regex.findall(scaffold)])[None,...].repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])

  if props is None:
    p = None
  elif len(props) == 1:
    # create a tensor for conditioning with a single property value
    p = torch.tensor([[props[0]]]).repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])
  else:
    # create a tensor for conditioning with multiple property values
    p = torch.tensor([props]).repeat(inference_config_dict["batch_size"], 1).unsqueeze(1).to(model_config_dict['device'])

  molecules = []
  for i in tqdm(range(gen_iter)):
          # create an input tensor by converting 'context' to a tensor of token indices,
          # repeat this batch times along the batch dimension
          x = torch.tensor([stoi[s] for s in regex.findall(inference_config_dict["context"])], dtype=torch.long)[None,...].repeat(inference_config_dict["batch_size"], 1).to(model_config_dict['device'])

          if ['X'] in inference_config_dict['context']:
            # call sample function to generate molecules conditioned on the input
            y = fill_mask(model, x, block_size, temperature=inference_config_dict["temp"], prop=p, scaffold=scaffold)
          else:
            y = sample(model, x, block_size, temperature=inference_config_dict["temp"], prop=p, scaffold=scaffold)

          # for each generated molecule:
          for gen_mol in y:
                  # convert generated molecule from list of integers to list of strings and concatenate to one string
                  completion = ''.join([itos[int(i)] for i in gen_mol])
                  # remove padding tokens
                  completion = completion.replace('<', '')
                  # convert the string representation of the molecule to an rdkit Mol object
                  mol = get_mol(completion)
                  # if an rdkit Mol object was created:
                  if mol:
                          # append the Mol object to the list
                          molecules.append(mol)
  # create dataframe where first column contains rdkit Mols and second column contins SMILES
  results = pd.DataFrame([{'molecule' : i, 'smiles': Chem.MolToSmiles(i)} for i in molecules])
  # iterate over each SMILES and ensure that equivalent molecules have same SMILES
  canon_smiles = [canonic_smiles(s) for s in results['smiles']]
  # create set of unique SMILES strings
  unique_smiles = list(set(canon_smiles))
  data = pd.read_csv(inference_config_dict["train_data"]) # load training data
  novel_ratio = check_novelty(unique_smiles, set(data['smiles'])) # calculate novelty ratio from generated SMILES and training SMILES
  results['qed'] = results['molecule'].apply(lambda x: QED.qed(x)) # quantitative estimate of drug-likeliness (QED)
  results['sas'] = results['molecule'].apply(lambda x: sascorer.calculateScore(x)) #synthetic accessibility score (SAS)
  results['logp'] = results['molecule'].apply(lambda x: Crippen.MolLogP(x)) #(measure of hydrophobicity)
  results['tpsa'] = results['molecule'].apply(lambda x: CalcTPSA(x)) #topological polar surface area (TPSA)
  results['validity'] = np.round(len(results)/(inference_config_dict["batch_size"]*gen_iter), 3)
  results['unique'] = np.round(len(unique_smiles)/len(results), 3)
  results['novelty'] = np.round(novel_ratio/100, 3)
  # save the dataframe as a csv file
  results.to_csv(inference_config_dict["save_path"], index = False)
  # print all evaluation metrics using function from moses package
  print(moses.get_all_metrics(list(results['smiles'].values), device=model_config_dict['device']))

In [ ]:
inference_config_dict = {
    # "model_params": train_config_dict["ckpt_path"],
    "model_params": BASE + 'checkpoints/test_6_21.pt',
    "train_data": train_config_dict["train_path"],
    # "desc_path": model_config_dict["desc_path"],
    "desc_path": BASE + 'checkpoints/descriptors_200k.yaml',
    "save_path": BASE + 'data/test_06_21_t.csv',
    "batch_size": 1,
    "gen_size": 10,
    "temp": 1,
    "context": "C1CCXCCC1",
    "scaffold": None,
    "props": []
}

In [ ]:
# run function to generate SMILES strings
generate_SMILES(
              model_config_dict = model_config_dict,
              inference_config_dict = inference_config_dict
              )